In [1]:
!python --version

Python 3.6.5 :: Anaconda, Inc.


In [2]:
#신경망구축에 필요한 tensorflow
#라이브러리와 numpy 그리고 시각화도구
#matplotlib를 import 한다.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

#Variable의 랜덤함수 시드를 설정한다.
tf.set_random_seed(777) #변경한 버전에서는 랜덤시드 사용 불가능해서 주석처리함.

In [3]:
print(tf.__version__)

1.15.0


In [4]:
#MinMaxScaler 정의 -> data를 0부터 1사이의 값으로 변환(normalize)
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [5]:
#train Parameter 상수값 설정
seq_length = 7 #7일 단위로 학습시키고 8일째를 예측
input_dim = 14 #input 데이터의 개수
hidden_dim = 16 #은닉층의 개수
output_dim = 1 # target label의 개수(배추가격)
learning_rate = 0.01
iterations = 10001

In [6]:
#값 설정이 끝나면 위에서 만든 데이터 셋을 불러온다. 그 후 데이터양에 따라 적절하게 Training Data와 Test Data로 분할한다.
#data load
#만들어둔 데이터셋을 Load한다.
xy = np.loadtxt('./apple_data.csv', delimiter=',')
xy = MinMaxScaler(xy) #Normalize
x = xy #전체 데이터(input)
y = xy[:,[-1]] #target Label(사과가격)

In [7]:
#build dataset
#Normalize가 끝난 데이터를 7일단위로 슬라이스 해서 3차원형태로 만든다.
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i: i+seq_length]
    _y = y[i+seq_length]
    dataX.append(_x)
    dataY.append(_y)
#train/test set 나누기
train_size = int(len(dataY) * 0.7) #train size = 70%
test_size = len(dataY) - train_size #test size = 30%
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

In [8]:
# 데이터 셋 분할이 끝났으면 해당 데이터를 받을 Placeholder와 인공신경망 구조인 LSTM을 구축한다.

#input placeholders
#input placeholder의 파라미터로 seq_length,input_dim를 넘겨준다.
X = tf.placeholder(tf.float32, [None, seq_length, input_dim]) #3차원 형태의 input placeholder
Y = tf.placeholder(tf.float32, [None, 1]) #2차원 형태의 output placeholder

In [9]:
#build a LSTM network (build Rnn)
#tensorflow를 이용하면 LSTM 구조를 cell 형태로 빠르고 쉽게 생성해준다.
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True,activation=tf.tanh)
#생성된 cell을 dynamic_rnn함수로 활성화 시켜준다. 파라미터로 생성된 cell과 input placeholder를 넘겨주고 리턴값으로 output을 받는다.
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)
#그 후 출력된 output을 fully_connected에 통과 시켜 최종 예측값을 리턴 받는다.
Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=tf.tanh)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.
